### Prepare space in flash

In [1]:
import os, subprocess

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220526_P190_laqueus_DiI_L-F11/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "ants")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Prepare mask

In [4]:
import nrrd
from probemapper.io import save_nifti, load_nifti
import numpy as np

In [5]:
raw = load_nifti(os.path.join(rootdir, "analysis/536BP_10um.nii"))
mask, _ = nrrd.read(os.path.join(rootdir, "analysis/slicer_536BP/Segmentation.seg.nrrd"))
mask = np.swapaxes(mask, 0, 2)
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "536BP_10um_masked.nii"), 10, 10, 10)

In [6]:
raw = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um.nii")
mask = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um_mask.nii.gz")
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10, 10, 10)

### Registration

In [7]:
from probemapper.registration import ANTSParams, ANTSStage, ANTSRegisteration

In [8]:
from probemapper.registration.helper import define_initial_transformation
import numpy as np

# aff = np.array([[ 1.0, 0, 0],
#                 [ 0, 1.0, 0],
#                 [ 0, 0, 1.0]], dtype='float')

aff = np.array([[ 1.        ,  0.        ,  0.        ],
                [ 0.        ,  0.93969262, -0.34202014],
                [ 0.        ,  0.34202014,  0.93969262]], dtype='float')

transl = np.array([0, 0, 0], dtype='float')
define_initial_transformation(aff, transl, os.path.join(tmpdir, "initial.mat"))

.mat file saved as /flash/ReiterU/tmp/P190/ants/initial.mat


In [9]:
P = ANTSParams(
    use_float=False,
    interpolation="Linear",
    histogram_matching=False,
    winsorize=[0.05, 1.0],
    init_transform_fx_to_mv=os.path.join(tmpdir, "initial.mat"),
    stages=[
        ANTSStage(
            transform="Affine",
            transform_args="0.1",
            metrics="MI",
            metrics_args="1,128,Regular,0.5",
            convergence="1000x1000x1000,1e-5,15",
            shrink_factors="8x4x2",
            smoothing_sigmas="3x2x1vox"
        ),
        ANTSStage(
            transform="SyN",
            transform_args="0.1,4.0,0.2",
            metrics="CC",
            metrics_args="1,6",
            convergence="500x500x100x30,1e-6,10",
            shrink_factors="8x4x2x1",
            smoothing_sigmas="3x2x1x0vox"
        )
    ]
)

In [10]:
R = ANTSRegisteration(P, ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin", num_threads=12)
R.set_fixed_image(os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10)
R.set_moving_image(os.path.join(tmpdir, "536BP_10um_masked.nii"), 10)
R.set_outputs(tmpdir)

In [11]:
R.run()

########################################
['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsRegistration', '--dimensionality', '3', '--initial-moving-transform', '/flash/ReiterU/tmp/P190/ants/initial.mat', '--interpolation', 'Linear', '--use-histogram-matching', '0', '--float', '0', '--verbose', '1', '--output', '[/flash/ReiterU/tmp/P190/ants/F2M_,/flash/ReiterU/tmp/P190/ants/Warped.nii.gz,/flash/ReiterU/tmp/P190/ants/InvWarped.nii.gz]', '--transform', 'Affine[0.1]', '--metric', 'MI[/flash/ReiterU/tmp/P190/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P190/ants/536BP_10um_masked.nii,1,128,Regular,0.5]', '--convergence', '[1000x1000x1000,1e-5,15]', '--shrink-factors', '8x4x2', '--smoothing-sigmas', '3x2x1vox', '--transform', 'SyN[0.1,4.0,0.2]', '--metric', 'CC[/flash/ReiterU/tmp/P190/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P190/ants/536BP_10um_masked.nii,1,6]', '--convergence', '[500x500x100x30,1e-6,10]', '--shrink-factors', '8x4x2x1', '--smoothing-sigmas', '3x2x1x0vox']


 2DIAGNOSTIC,     5, -9.212179405122e-02, 1.797693134862e+308, 2.8234e+00, 5.3135e-02,
 2DIAGNOSTIC,     6, -9.212957025055e-02, 1.797693134862e+308, 2.8745e+00, 5.1164e-02,
 2DIAGNOSTIC,     7, -9.212615581549e-02, 1.797693134862e+308, 2.9189e+00, 4.4349e-02,
 2DIAGNOSTIC,     8, -9.215025819613e-02, 1.797693134862e+308, 2.9626e+00, 4.3662e-02,
 2DIAGNOSTIC,     9, -9.218575306643e-02, 1.797693134862e+308, 3.0071e+00, 4.4574e-02,
 2DIAGNOSTIC,    10, -9.237586385885e-02, 1.797693134862e+308, 3.0600e+00, 5.2890e-02,
 2DIAGNOSTIC,    11, -9.239787072342e-02, 1.797693134862e+308, 3.1076e+00, 4.7582e-02,
 2DIAGNOSTIC,    12, -9.243848516020e-02, 1.797693134862e+308, 3.1530e+00, 4.5414e-02,
 2DIAGNOSTIC,    13, -9.248965379829e-02, 1.797693134862e+308, 3.2082e+00, 5.5150e-02,
 2DIAGNOSTIC,    14, -9.254708411913e-02, 1.797693134862e+308, 3.2565e+00, 4.8341e-02,
 2DIAGNOSTIC,    15, -9.270554158556e-02, 3.092047070625e-04, 3.3102e+00, 5.3689e-02,
 2DIAGNOSTIC,    16, -9.404018000893e-02, 4.

 2DIAGNOSTIC,    17, -3.954783584042e-01, 3.268480845462e-04, 1.7635e+01, 2.8217e-01,
 2DIAGNOSTIC,    18, -3.962630866443e-01, 3.544206502996e-04, 1.7962e+01, 3.2678e-01,
 2DIAGNOSTIC,    19, -3.965319909959e-01, 3.714432797075e-04, 1.8244e+01, 2.8150e-01,
 2DIAGNOSTIC,    20, -3.980293176805e-01, 4.092014956505e-04, 1.8577e+01, 3.3362e-01,
 2DIAGNOSTIC,    21, -3.982920425642e-01, 4.336803854155e-04, 1.8910e+01, 3.3303e-01,
 2DIAGNOSTIC,    22, -3.983873373429e-01, 4.418187381160e-04, 1.9210e+01, 2.9971e-01,
 2DIAGNOSTIC,    23, -3.986559261948e-01, 4.395341538056e-04, 1.9568e+01, 3.5808e-01,
 2DIAGNOSTIC,    24, -3.988143796007e-01, 4.274579071017e-04, 1.9875e+01, 3.0685e-01,
 2DIAGNOSTIC,    25, -3.993538045776e-01, 4.132103560510e-04, 2.0265e+01, 3.8999e-01,
 2DIAGNOSTIC,    26, -3.997200999900e-01, 3.934180320507e-04, 2.0749e+01, 4.8420e-01,
 2DIAGNOSTIC,    27, -3.997107358351e-01, 3.707221479526e-04, 2.1047e+01, 2.9754e-01,
 2DIAGNOSTIC,    28, -3.999138663861e-01, 3.4166649262

 2DIAGNOSTIC,     4, -4.251938169098e-01, 1.797693134862e+308, 6.1973e+01, 1.7808e+00,
 2DIAGNOSTIC,     5, -4.252054664935e-01, 1.797693134862e+308, 6.3710e+01, 1.7372e+00,
 2DIAGNOSTIC,     6, -4.252363043505e-01, 1.797693134862e+308, 6.5556e+01, 1.8461e+00,
 2DIAGNOSTIC,     7, -4.252845476016e-01, 1.797693134862e+308, 6.7351e+01, 1.7947e+00,
 2DIAGNOSTIC,     8, -4.254095989211e-01, 1.797693134862e+308, 6.9277e+01, 1.9261e+00,
 2DIAGNOSTIC,     9, -4.254737414010e-01, 1.797693134862e+308, 7.1773e+01, 2.4964e+00,
 2DIAGNOSTIC,    10, -4.255638961178e-01, 1.797693134862e+308, 7.3540e+01, 1.7665e+00,
 2DIAGNOSTIC,    11, -4.272012354375e-01, 1.797693134862e+308, 7.5979e+01, 2.4397e+00,
 2DIAGNOSTIC,    12, -4.281537585259e-01, 1.797693134862e+308, 7.7867e+01, 1.8878e+00,
 2DIAGNOSTIC,    13, -4.282967972718e-01, 1.797693134862e+308, 8.0026e+01, 2.1586e+00,
 2DIAGNOSTIC,    14, -4.283195573983e-01, 1.797693134862e+308, 8.1935e+01, 1.9090e+00,
 2DIAGNOSTIC,    15, -4.284475826902e-01, 3

 1DIAGNOSTIC,    38, -8.013967966446e-01, 2.050409923751e-04, 8.3152e+01, 2.1421e+00,
 1DIAGNOSTIC,    39, -8.023591407363e-01, 1.950012566655e-04, 8.5301e+01, 2.1496e+00,
 1DIAGNOSTIC,    40, -8.032806204711e-01, 1.859955480554e-04, 8.7532e+01, 2.2302e+00,
 1DIAGNOSTIC,    41, -8.042063791106e-01, 1.779527715213e-04, 8.9666e+01, 2.1344e+00,
 1DIAGNOSTIC,    42, -8.051080385940e-01, 1.705122986332e-04, 9.1852e+01, 2.1864e+00,
 1DIAGNOSTIC,    43, -8.060876475857e-01, 1.645878914544e-04, 9.3943e+01, 2.0902e+00,
 1DIAGNOSTIC,    44, -8.070612279520e-01, 1.599084644743e-04, 9.6009e+01, 2.0662e+00,
 1DIAGNOSTIC,    45, -8.080822282492e-01, 1.566495773141e-04, 9.8136e+01, 2.1272e+00,
 1DIAGNOSTIC,    46, -8.091332825814e-01, 1.547995800858e-04, 1.0028e+02, 2.1445e+00,
 1DIAGNOSTIC,    47, -8.101936899677e-01, 1.540020659597e-04, 1.0242e+02, 2.1365e+00,
 1DIAGNOSTIC,    48, -8.112709101362e-01, 1.535383103955e-04, 1.0460e+02, 2.1816e+00,
 1DIAGNOSTIC,    49, -8.122674045164e-01, 1.5294852913

 1DIAGNOSTIC,   134, -8.399236385989e-01, 2.388213696088e-06, 2.8724e+02, 2.2943e+00,
 1DIAGNOSTIC,   135, -8.398890162762e-01, 2.417581073293e-06, 2.8937e+02, 2.1217e+00,
 1DIAGNOSTIC,   136, -8.399993629252e-01, 2.357788792831e-06, 2.9151e+02, 2.1408e+00,
 1DIAGNOSTIC,   137, -8.400924402529e-01, 2.572451321734e-06, 2.9359e+02, 2.0788e+00,
 1DIAGNOSTIC,   138, -8.400331523256e-01, 2.674205021119e-06, 2.9569e+02, 2.1057e+00,
 1DIAGNOSTIC,   139, -8.401226931279e-01, 2.646730323011e-06, 2.9784e+02, 2.1518e+00,
 1DIAGNOSTIC,   140, -8.402105111162e-01, 2.729916525142e-06, 2.9989e+02, 2.0461e+00,
 1DIAGNOSTIC,   141, -8.401593114752e-01, 2.690018725781e-06, 3.0197e+02, 2.0844e+00,
 1DIAGNOSTIC,   142, -8.402552126296e-01, 2.480400517104e-06, 3.0423e+02, 2.2531e+00,
 1DIAGNOSTIC,   143, -8.402190835069e-01, 2.182891738424e-06, 3.0634e+02, 2.1092e+00,
 1DIAGNOSTIC,   144, -8.402893444135e-01, 2.124445131447e-06, 3.0848e+02, 2.1473e+00,
 1DIAGNOSTIC,   145, -8.403669729680e-01, 1.9731826074

 1DIAGNOSTIC,    55, -8.700912254208e-01, 6.598444119599e-06, 1.2397e+03, 1.4944e+01,
 1DIAGNOSTIC,    56, -8.700575594765e-01, 5.568399073438e-06, 1.2543e+03, 1.4576e+01,
 1DIAGNOSTIC,    57, -8.701508159110e-01, 5.415775898045e-06, 1.2702e+03, 1.5931e+01,
 1DIAGNOSTIC,    58, -8.701108344951e-01, 4.552932412770e-06, 1.2869e+03, 1.6731e+01,
 1DIAGNOSTIC,    59, -8.701420939753e-01, 4.206336001792e-06, 1.3019e+03, 1.4935e+01,
 1DIAGNOSTIC,    60, -8.701576297787e-01, 3.523572619905e-06, 1.3194e+03, 1.7538e+01,
 1DIAGNOSTIC,    61, -8.701845443421e-01, 3.318444821047e-06, 1.3348e+03, 1.5420e+01,
 1DIAGNOSTIC,    62, -8.701408573661e-01, 2.477354525049e-06, 1.3505e+03, 1.5675e+01,
 1DIAGNOSTIC,    63, -8.702308748858e-01, 2.493745801191e-06, 1.3657e+03, 1.5185e+01,
 1DIAGNOSTIC,    64, -8.701823222343e-01, 1.895050237542e-06, 1.3814e+03, 1.5724e+01,
 1DIAGNOSTIC,    65, -8.702064539743e-01, 1.826086334738e-06, 1.3977e+03, 1.6333e+01,
 1DIAGNOSTIC,    66, -8.702188105630e-01, 1.5108596619

 1DIAGNOSTIC,    78, -8.831640953271e-01, 1.717842812804e-06, 9.7866e+03, 1.0241e+02,
 1DIAGNOSTIC,    79, -8.831605877406e-01, 1.596207343338e-06, 9.8906e+03, 1.0400e+02,
 1DIAGNOSTIC,    80, -8.831923490432e-01, 1.632427233892e-06, 9.9963e+03, 1.0568e+02,
 1DIAGNOSTIC,    81, -8.831872016578e-01, 1.523088651374e-06, 1.0102e+04, 1.0556e+02,
 1DIAGNOSTIC,    82, -8.832183254635e-01, 1.559591666660e-06, 1.0207e+04, 1.0554e+02,
 1DIAGNOSTIC,    83, -8.832131677852e-01, 1.447990943031e-06, 1.0313e+04, 1.0606e+02,
 1DIAGNOSTIC,    84, -8.832427732624e-01, 1.473891095978e-06, 1.0417e+04, 1.0345e+02,
 1DIAGNOSTIC,    85, -8.832371007732e-01, 1.359776977100e-06, 1.0519e+04, 1.0202e+02,
 1DIAGNOSTIC,    86, -8.832659151944e-01, 1.383700470864e-06, 1.0625e+04, 1.0607e+02,
 1DIAGNOSTIC,    87, -8.832615638709e-01, 1.278582472661e-06, 1.0733e+04, 1.0823e+02,
 1DIAGNOSTIC,    88, -8.832561738707e-01, 1.187046279731e-06, 1.0840e+04, 1.0702e+02,
 1DIAGNOSTIC,    89, -8.832863683370e-01, 1.1132519568

### Copy files to bucket

In [12]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P190/ants', 'deigo:/bucket/ReiterU/DBS/data/20220526_P190_laqueus_DiI_L-F11/analysis'], returncode=0)